# Topic modelling of news headlines for prediction of news category¶

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier and assume 7 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import wordcloud

import re

import pyLDAvis.lda_model

## Load train set

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...


## Load German stopwords

In [4]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [5]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

## Extract features from 'title' 

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words=stop_words)

In [8]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [9]:
dtm

<67513x24939 sparse matrix of type '<class 'numpy.int64'>'
	with 350924 stored elements in Compressed Sparse Row format>

In [10]:
from sklearn.decomposition import LatentDirichletAllocation

In [11]:
lda = LatentDirichletAllocation(n_components=7, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Analyse extracted features

In [12]:
len(cv.get_feature_names_out())

24939

In [13]:
cv.get_feature_names_out()

array(['00', '000', '007', ..., 'или', 'российские', 'украинцы'],
      dtype=object)

In [14]:
len(lda.components_)

7

In [15]:
lda.components_

array([[1.43027242e-01, 1.43024873e-01, 1.42857397e-01, ...,
        1.42857602e-01, 1.42857338e-01, 1.42857539e-01],
       [1.42857251e-01, 2.48500645e+02, 1.44049478e-01, ...,
        1.42857668e-01, 1.42857366e-01, 1.42857595e-01],
       [1.43148393e-01, 4.48722291e+01, 1.42857443e-01, ...,
        1.14298451e+00, 2.14285583e+00, 1.42857623e-01],
       ...,
       [3.14235903e+00, 2.34543085e-01, 1.42857455e-01, ...,
        1.42857728e-01, 1.42857391e-01, 1.42857646e-01],
       [1.42857223e-01, 3.15093314e+01, 1.42857353e-01, ...,
        1.42857533e-01, 1.42857308e-01, 1.42857479e-01],
       [1.42893606e-01, 1.14313352e+02, 2.14115857e+00, ...,
        1.14272726e+00, 1.42857386e-01, 2.14285450e+00]])

In [16]:
len(lda.components_[0])

24939

### Show most important words of first extracted topic

In [17]:
first_topic = lda.components_[0]

In [18]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([22881,  5717, 12715, ..., 21919, 21221,  2841], dtype=int64)

In [19]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [20]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

wahrheit
afd
bundesliga
bayern
wahl
ex
cdu
us
trump
berlin


### Show most important words of first extracted topic

In [21]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['haft', 'letzte', 'generation', 'spd', 'gericht', 'wahrheit', 'afd', 'bundesliga', 'bayern', 'wahl', 'ex', 'cdu', 'us', 'trump', 'berlin']


Die TOP-15 Wörter für das Thema #1
['millionen', 'frauen', 'neue', 'prozent', 'biden', 'weg', '000', 'deutschland', 'gas', 'inflation', 'deutsche', 'us', 'iran', 'regierung', 'menschen']


Die TOP-15 Wörter für das Thema #2
['frauen', 'nordkorea', 'macron', 'steingart', 'gabor', 'lauterbach', 'usa', 'tote', 'dfb', 'gastbeitrag', 'polen', 'eu', 'frankreich', 'wm', 'deutschland']


Die TOP-15 Wörter für das Thema #3
['scholz', 'warnt', 'analyse', 'chinas', 'news', 'ampel', 'afd', 'kritik', 'lage', 'taiwan', 'marktbericht', 'pandemie', 'invasion', 'china', 'corona']


Die TOP-15 Wörter für das Thema #4
['online', 'wegen', 'fdp', 'tot', 'scholz', 'analyse', 'grünen', 'reitz', 'ampel', 'ulrich', 'stream', 'bundestag', 'türkei', 'nord', 'streit']


Die TOP-15 Wörter für das Thema #5
['nachrichten', 'usa', 'russen', 'r

### Get topics with highest probability for news items in train data

In [22]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 7)

In [23]:
topic_results[0].round(5)

array([0.01791, 0.01791, 0.01786, 0.89272, 0.01787, 0.01786, 0.01786])

In [24]:
topic_results[0].argmax()

3

In [25]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [26]:
train.head()

,date,title,description,category,text,source,title_description_text,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,3
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,5
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,4
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,5
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,4


## Visualize model

In [27]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, dtm, cv, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
5      -15.867260    5.233803       1        1  18.393703
0       70.422318  -89.428223       2        1  15.805916
1      -54.471935 -116.444168       3        1  13.920579
3     -101.887115   99.503708       4        1  13.290075
2       22.866714  126.841034       5        1  13.161780
6     -140.708176  -22.340219       6        1  12.824983
4      108.877716   32.309097       7        1  12.602963, topic_info=              Term         Freq        Total Category  logprob  loglift
21398      ukraine  4471.000000  4471.000000  Default  30.0000  30.0000
12075        krieg  2487.000000  2487.000000  Default  29.0000  29.0000
4661   deutschland  1489.000000  1489.000000  Default  28.0000  28.0000
4303        corona  1152.000000  1152.000000  Default  27.0000  27.0000
17650     russland  1590.000000  1590.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
6470            ex   137.478786   661.859782   Topic7  -5.7797   0.4997
19362          spd   128.237486   488.908536   Topic7  -5.8493   0.7329
11730    kommentar   115.421560   308.631663   Topic7  -5.9546   1.0877
17650     russland   130.604726  1590.387747   Topic7  -5.8310  -0.4283
4659     deutscher   102.405383   210.855214   Topic7  -6.0742   1.3490

[454 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
1          1  0.070146        000
1          3  0.561170        000
1          4  0.007257        000
1          5  0.101591        000
1          6  0.258815        000
...      ...       ...        ...
24740      3  0.833690         öl
24740      4  0.008173         öl
24740      5  0.008173         öl
24740      7  0.138948         öl
24780      1  0.993430  überblick

[891 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 2, 4, 3, 7, 5])

## Save train set with topics and dump model

In [28]:
#Save result to csv
train.to_csv('train_lda_topics_7.csv')

In [29]:
# Dump countvectorizer
joblib.dump(cv, 'cv.jl')

['cv.jl']

In [30]:
# Dump LDA model
joblib.dump(lda, 'lda_model_7_topics.jl')

['lda_model_7_topics.jl']